In [37]:
import deepchem as dc
from deepchem.molnet import load_delaney, load_freesolv, load_qm7, load_lipo
import torch
import torch.nn as nn
import gpytorch
import numpy as np

# 1. Load data
# tasks, datasets, transformers = load_freesolv(featurizer='ECFP')
tasks, datasets, transformers = load_lipo()
# tasks, datasets, transformers = load_delaney(featurizer='ECFP')

train_dc, valid_dc, test_dc = datasets

n_tasks = len(tasks)               # usually 1
# n_features = train_dc.X.shape[1]      # ECFP length

[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerator
[13:52:01] DEPRECATION WARNING: please use MorganGenerat

In [39]:
print(train_dc.X)
print(valid_dc.X.shape)
print(test_dc.X.shape)
print(train_dc.y.shape)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(420, 1024)
(420, 1024)
(3360, 1)


In [30]:
train_dc

<DiskDataset X.shape: (np.int64(5470), np.int64(23), np.int64(23)), y.shape: (np.int64(5470), np.int64(1)), w.shape: (np.int64(5470), np.int64(1)), task_names: ['u0_atom']>

In [3]:
# 2. Define your PyTorch model
class MyTorchRegressor(nn.Module):
    def __init__(self, n_features, n_tasks):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, n_tasks)
        )

    def forward(self, x):
        # x: (batch_size, n_features)
        return self.net(x)  # (batch_size, n_tasks)


torch_model = MyTorchRegressor(n_features, n_tasks)

# 3. DeepChem loss (NOT torch.nn.MSELoss)
loss = dc.models.losses.L2Loss()

# 4. Wrap in TorchModel
dc_model = dc.models.TorchModel(
    model=torch_model,
    loss=loss,
    output_types=['prediction'],   # normal regression output
    batch_size=64,
    learning_rate=1e-3,            # DeepChem will create its own Adam optimizer
    mode='regression'
)

# 5. Train
dc_model.fit(train_dc, nb_epoch=10)

# 6. Evaluate
metric = dc.metrics.Metric(dc.metrics.mean_squared_error)
valid_scores = dc_model.evaluate(valid_dc, [metric])
test_scores = dc_model.evaluate(test_dc, [metric])

print("Validation MSE:", valid_scores[metric.name])
print("Test MSE:", test_scores[metric.name])

Validation MSE: 0.6796141123379799
Test MSE: 0.7320434542583424


In [4]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)

        input_dim = train_x.shape[-1]  # number of features

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims=input_dim)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [5]:
class GPyTorchRegressor(nn.Module):
    def __init__(self, train_x, train_y):
        super().__init__()

        # Register training data (for ExactGP)
        self.register_buffer("train_x", train_x)
        self.register_buffer("train_y", train_y)

        # Likelihood + GP model
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()
        self.gp_model = ExactGPModel(self.train_x, self.train_y, self.likelihood)

    def forward(self, x):
        """
        Behaves like a normal PyTorch regressor:
        input:  x  (batch_size, input_dim)
        output: mean prediction (batch_size, 1)
        """
        self.gp_model.eval()
        self.likelihood.eval()
        with torch.no_grad():
            pred_dist = self.likelihood(self.gp_model(x))
            mean = pred_dist.mean  # (batch_size,)
            return mean.unsqueeze(-1)  # (batch_size, 1)

    def predict_interval(self, x, alpha: float = 0.05):
        """
        Returns mean, lower, upper for a (1 - alpha) confidence interval.
        Default alpha=0.05 -> 95% interval.
        """
        self.gp_model.eval()
        self.likelihood.eval()
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            pred_dist = self.likelihood(self.gp_model(x))
            mean = pred_dist.mean          # (batch_size,)
            std = pred_dist.stddev         # (batch_size,)

            # z-score for (1 - alpha) CI, e.g. 1.96 for 95%
            z = 1.96 if alpha == 0.05 else torch.distributions.Normal(0, 1).icdf(
                torch.tensor(1 - alpha / 2, device=x.device)
            )

            lower = mean - z * std
            upper = mean + z * std
            return mean, lower, upper      # each is (batch_size,)

In [6]:
class GPTrainer:
    def __init__(
        self,
        model: GPyTorchRegressor,
        train_dataset: dc.data.NumpyDataset,
        lr: float = 0.1,
        num_iters: int = 100,
        device: str = "cpu",
        log_interval: int = 10,
    ):
        self.model = model.to(device)
        self.device = device
        self.num_iters = num_iters
        self.log_interval = log_interval

        # Convenience handles
        self.likelihood = self.model.likelihood
        self.gp = self.model.gp_model

        # Convert DeepChem train dataset to torch tensors
        self.train_x, self.train_y = self._dc_to_torch(train_dataset)

        # ExactMarginalLogLikelihood loss
        self.mll = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.gp)

        # Optimizer (only on GP parameters)
        self.optimizer = torch.optim.Adam(self.gp.parameters(), lr=lr)

    def _dc_to_torch(self, dataset: dc.data.NumpyDataset):
        """Convert DeepChem NumpyDataset to torch tensors on the correct device."""
        X = torch.from_numpy(dataset.X).float().to(self.device)
        # dataset.y shape: (N, n_tasks). For Delaney n_tasks=1 so squeeze.
        y = torch.from_numpy(dataset.y).float().to(self.device)
        if y.ndim == 2 and y.shape[1] == 1:
            y = y.squeeze(-1)
        return X, y

    def train(self):
        """Standard Exact GP training loop."""
        self.model.train()
        self.likelihood.train()

        for i in range(1, self.num_iters + 1):
            self.optimizer.zero_grad()
            output = self.gp(self.train_x)        # MultivariateNormal
            loss = -self.mll(output, self.train_y)
            loss.backward()
            self.optimizer.step()

            if i % self.log_interval == 0 or i == 1 or i == self.num_iters:
                print(f"[Iter {i:03d}] Loss: {loss.item():.4f}")

    def _predict_mean_tensor(self, dataset):
        """Internal helper: get mean predictions as tensor (N,) on a dataset."""
        self.model.eval()
        self.likelihood.eval()

        X, _ = self._dc_to_torch(dataset)
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            mean = self.model(X).squeeze(-1)  # (N,)
        return mean

    def evaluate_mse(self, dataset: dc.data.NumpyDataset) -> float:
        """Compute mean squared error on a DeepChem dataset."""
        self.model.eval()
        self.likelihood.eval()

        X, y_true = self._dc_to_torch(dataset)
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            mean = self.model(X).squeeze(-1)  # (N,)

        mse = torch.mean((mean - y_true) ** 2).item()
        return mse

    def predict_mean(self, dataset: dc.data.NumpyDataset) -> np.ndarray:
        """Return mean predictions for a dataset as a NumPy array (N,)."""
        mean = self._predict_mean_tensor(dataset)
        return mean.cpu().numpy()

    def predict_interval(
        self,
        dataset: dc.data.NumpyDataset,
        alpha: float = 0.05,
    ):
        """
        Return (mean, lower, upper) as NumPy arrays, each shape (N,).
        """
        self.model.eval()
        self.likelihood.eval()

        X, _ = self._dc_to_torch(dataset)
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            mean, lower, upper = self.model.predict_interval(X, alpha=alpha)

        return (
            mean.cpu().numpy(),
            lower.cpu().numpy(),
            upper.cpu().numpy(),
        )

In [7]:
# Convert DeepChem train data to torch tensors for model construction
X_train_torch = torch.from_numpy(train_dc.X).float()
y_train_torch = torch.from_numpy(train_dc.y[:, 0]).float()  # Delaney: 1 task

# Build model
gp_model_wrapper = GPyTorchRegressor(X_train_torch, y_train_torch)

# Build trainer
trainer = GPTrainer(
    model=gp_model_wrapper,
    train_dataset=train_dc,
    lr=0.1,
    num_iters=100,
    device="cpu",
    log_interval=10,
)

# Train GP
trainer.train()

# Evaluate
valid_mse = trainer.evaluate_mse(valid_dc)
test_mse = trainer.evaluate_mse(test_dc)
print("Validation MSE:", valid_mse)
print("Test MSE:", test_mse)

[Iter 001] Loss: 1.4197
[Iter 010] Loss: 1.3223
[Iter 020] Loss: 1.0988
[Iter 030] Loss: 0.9454
[Iter 040] Loss: 0.8729
[Iter 050] Loss: 0.8296
[Iter 060] Loss: 0.8158
[Iter 070] Loss: 0.7857
[Iter 080] Loss: 0.7745
[Iter 090] Loss: 0.7625
[Iter 100] Loss: 0.7648
Validation MSE: 0.8327946066856384
Test MSE: 0.840823769569397


In [8]:
# Get uncertainty intervals on test set
mean_test, lower_test, upper_test = trainer.predict_interval(test_dc, alpha=0.05)
print("First 5 test mean:", mean_test[:5])
print("First 5 lower:", lower_test[:5])
print("First 5 upper:", upper_test[:5])
print("First 5 gt:", test_dc.y[:5])

First 5 test mean: [-0.9030192   0.0697577  -0.27833986 -1.537761   -0.6202069 ]
First 5 lower: [-2.07172   -1.2944871 -1.708732  -2.8169284 -1.8074324]
First 5 upper: [ 0.2656815   1.4340025   1.1520523  -0.25859356  0.5670187 ]
First 5 gt: [[-1.60114461]
 [ 0.20848251]
 [-0.01602738]
 [-2.82191713]
 [-0.52891635]]


In [33]:
class GPyTorchMultitaskRegressor(nn.Module):
    """
    - forward(x) -> mean predictions, shape (N, T)
    - predict_interval(x, alpha) -> (mean, lower, upper), each (N, T)
    """
    def __init__(self, train_x, train_y):
        super().__init__()

        # store training data
        self.register_buffer("train_x", train_x)   # (N, D)
        self.register_buffer("train_y", train_y)   # (N, T)
        self.num_tasks = train_y.shape[-1]

        # multitask likelihood
        self.likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(
            num_tasks=self.num_tasks
        )

        # multitask GP model
        self.gp_model = MultitaskExactGPModel(self.train_x, self.train_y, self.likelihood)

    def forward(self, x):
        """
        Returns mean prediction (N, T) so it still “looks like”
        a normal multitask regressor.
        """
        self.gp_model.eval()
        self.likelihood.eval()
        with torch.no_grad():
            dist = self.likelihood(self.gp_model(x))   # MultitaskMultivariateNormal
            mean = dist.mean                           # (N, T)
            return mean

    def predict_interval(self, x, alpha: float = 0.05):
        """
        Returns (mean, lower, upper), each shape (N, T).
        """
        self.gp_model.eval()
        self.likelihood.eval()
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            dist = self.likelihood(self.gp_model(x))
            mean = dist.mean          # (N, T)
            std = dist.stddev         # (N, T)

            if alpha == 0.05:
                z = 1.96
            else:
                z = torch.distributions.Normal(0, 1).icdf(
                    torch.tensor(1 - alpha / 2, device=x.device)
                )

            lower = mean - z * std
            upper = mean + z * std
            return mean, lower, upper

In [34]:
# Convert DeepChem train data to torch tensors for model construction
X_train_torch = torch.from_numpy(train_dc.X).float()
y_train_torch = torch.from_numpy(train_dc.y).float()

# Build model
gp_model_wrapper = GPyTorchMultitaskRegressor(X_train_torch, y_train_torch)

# Build trainer
trainer = GPTrainer(
    model=gp_model_wrapper,
    train_dataset=train_dc,
    lr=0.1,
    num_iters=100,
    device="cpu",
    log_interval=10,
)

# Train GP
trainer.train()

# Evaluate
valid_mse = trainer.evaluate_mse(valid_dc)
test_mse = trainer.evaluate_mse(test_dc)
print("Validation MSE:", valid_mse)
print("Test MSE:", test_mse)

# Get uncertainty intervals on test set
mean_test, lower_test, upper_test = trainer.predict_interval(test_dc, alpha=0.05)
print("First 5 test mean:", mean_test[:5])
print("First 5 lower:", lower_test[:5])

NameError: name 'MultitaskExactGPModel' is not defined